# Master SQL Fundamentals Effortlessly as a Pandas User
## There was a time it was the other way around
![](images/unsplash.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@itssammoqadam?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Sam Moqadam</a>
        on 
        <a href='https://unsplash.com/?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Unsplash</a>
    </strong>
</figcaption>

### Motivation

When Pandas package gained public exposure in 2009, SQL had been dominating the data world since 1974. Pandas came with an attractive set of features such as in-built visualization, flexible data handling and became an ultimate data exploration tool. As it started gaining mass popularity, many courses and resources emerged teaching Pandas and comparing it to SQL.

Flash-forward to 2021, people are now getting introduced to the Pandas package *first* rather than the universal data language - SQL. Even though SQL is as popular as ever, the flexibility and multi-functionality of Pandas is making it the first choice for beginner data scientists.

Then, why do you need SQL if you know Pandas?

Even though Pandas may seem a better choice, SQL still plays a crucial role in day-to-day job of a data scientist. In fact, SQL is the second most in-demand and the third most growing programming language for data science (see [here](https://towardsdatascience.com/the-most-in-demand-skills-for-data-scientists-in-2021-4b2a808f4005)). So, it is a must to add SQL to your CV if you want to get a job in the field. And knowing Pandas, learning SQL should be a breeze, as you will see in this article.

### Connecting to a Database

Setting up an SQL workspace and connecting to a database can be a real pain in the neck. First, you need to install your favorite SQL flavor (PostgreSQL, MySQL, etc.) and download an SQL IDE too. Doing all those here would deviate us from the purpose of the article, so we will use a shortcut. 

Specifically, we will directly run SQL queries in a Jupyter Notebook without any additional steps. All we need to do is install the `ipython-sql` package using pip:

```python
pip install ipython-sql
```

After the installation is done, start a new Jupyter session and run this command in the notebook:

```python
%load_ext sql
```
and we are all set!

To illustrate how basic SQL statements work, we will be using the Chinook database which can be downloaded [here](https://www.sqlitetutorial.net/sqlite-sample-database/). The database has 11 tables, each of which has its own name. 

To retrieve the data stored in this database's tables, run this command:

In [1]:
%load_ext sql
%sql sqlite:///data/chinook.db

The statement starts with `%sql` in-line magic command that tells the notebook interpreter we will be running SQL commands. It is followed by the path that our downloaded Chinook database is in. The valid paths should always start with `sqlite:///` prefix for SQLite databases. Above, we are connecting to the database that is stored in the 'data' folder of the current directory. If you want to pass an absolute path, the prefix should be with 4 forward slashes - `sqlite:////`

If you wish to connect to a different database flavor, you can refer to this [excellent article](https://towardsdatascience.com/heres-how-to-run-sql-in-jupyter-notebooks-f26eb90f3259).

### Taking a First Look at the Tables

The first thing we always do in Pandas is use the `.head()` function. Let's learn how to do that in SQL:

In [2]:
%%sql

SELECT * FROM customers
LIMIT 5

 * sqlite:///data/chinook.db
Done.


CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


The first keyword in the above query is `SELECT`. It is equivalent to the brackets operator in Pandas where we select specific columns. But, the SELECT keyword is followed by a * (asterisk). * is an SQL operator that tells to select everything (all rows and columns) from a table specified after the `FROM` keyword. LIMIT is used to minimize the output returned. So, the above query is equivalent to `df.head()` function.

If you don't want to select all columns, you can specify one or more column names after the SELECT keyword:

In [3]:
%%sql

SELECT Name, Composer, UnitPrice
FROM tracks
LIMIT 10

 * sqlite:///data/chinook.db
Done.


Name,Composer,UnitPrice
For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",0.99
Balls to the Wall,None,0.99
Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",0.99
Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",0.99
Princess of the Dawn,Deaffy & R.A. Smith-Diesel,0.99
Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",0.99
Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",0.99
Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",0.99
Snowballed,"Angus Young, Malcolm Young, Brian Johnson",0.99
Evil Walks,"Angus Young, Malcolm Young, Brian Johnson",0.99


The equivalent Pandas operation is 
```python
tracks[['Name', 'Composer', 'UnitPrice']].head(10)
```

Another useful keyword in SQL is `DISTINCT`. Adding this keyword before any column name returns its unique values:

In [4]:
%%sql

SELECT DISTINCT FirstName
FROM employees

-- equivalent to `.value_counts()`

 * sqlite:///data/chinook.db
Done.


FirstName
Andrew
Nancy
Jane
Margaret
Steve
Michael
Robert
Laura


Comments in SQL are written with double dashes

### Counting the Number of Rows

Just like Pandas has `.shape` attribute on its DataFrames, SQL has a `COUNT` function to display the number of rows in a table:

In [5]:
%%sql

SELECT COUNT(*) FROM tracks

 * sqlite:///data/chinook.db
Done.


COUNT(*)
3503


It is also possible to pass a column name to COUNT:

In [6]:
%sql SELECT COUNT(FirstName) FROM customers

 * sqlite:///data/chinook.db
Done.


COUNT(FirstName)
59


But the output would be the same as COUNT(\*). A more useful info would be counting the number of unique values in a particular column. We can do this by adding the DISTINCT keyword into COUNT:

In [7]:
%%sql 

SELECT COUNT(DISTINCT FirstName) FROM employees

-- equivalent to `employees['FirstName'].nunique()`

 * sqlite:///data/chinook.db
Done.


COUNT(DISTINCT FirstName)
8


### Filtering Results With WHERE Clauses

Just looking and counting rows is pretty lame. Let's see how we can display rows based on conditions.

First, let's look at the songs which cost more than a dollar:

In [8]:
%%sql

SELECT * FROM tracks
WHERE UnitPrice > 1.0
LIMIT 10

--tracks[tracks['UnitPrice' > 1.0]]

 * sqlite:///data/chinook.db
Done.


TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
2819,Battlestar Galactica: The Story So Far,226,3,18,None,2622250,490750393,1.99
2820,Occupation / Precipice,227,3,19,None,5286953,1054423946,1.99
2821,"Exodus, Pt. 1",227,3,19,None,2621708,475079441,1.99
2822,"Exodus, Pt. 2",227,3,19,None,2618000,466820021,1.99
2823,Collaborators,227,3,19,None,2626626,483484911,1.99
2824,Torn,227,3,19,None,2631291,495262585,1.99
2825,A Measure of Salvation,227,3,18,None,2563938,489715554,1.99
2826,Hero,227,3,18,None,2713755,506896959,1.99
2827,Unfinished Business,227,3,18,None,2622038,528499160,1.99
2828,The Passage,227,3,18,None,2623875,490375760,1.99


Conditional statements are written after in the WHERE clause which always comes after FROM and before the LIMIT keywords. Using conditionals are pretty similar to how we do it in Pandas. You can also use the COUNT function when using conditionals. For example, let's see the number of songs that are priced between 1 and 10 dollars:

In [9]:
%%sql

SELECT COUNT(*) FROM tracks
WHERE UnitPrice > 1.0 AND UnitPrice < 10

-- tracks[(tracks['UnitPrice'] > 1) & (tracks['UnitPrice'] < 10)]

 * sqlite:///data/chinook.db
Done.


COUNT(*)
213


Above we chained two conditions with the boolean operator AND. Other boolean operators are the same in SQL. 

Now, let's see all the invoices that has Paris or Berlin as a billing city:

In [10]:
%%sql

SELECT BillingAddress, BillingCity, Total
FROM invoices
WHERE BillingCity = 'Paris' OR BillingCity = 'Berlin'
LIMIT 5

--invoices[(invoices['BillingCity'] == 'Paris') &
--         (invoices['BillingCity'] == 'Berlin')]

 * sqlite:///data/chinook.db
Done.


BillingAddress,BillingCity,Total
Barbarossastraße 19,Berlin,1.98
"8, Rue Hanovre",Paris,1.98
"8, Rue Hanovre",Paris,13.86
Tauentzienstraße 8,Berlin,1.98
Barbarossastraße 19,Berlin,3.96


Equality operator in SQL requires only one '=' (equal) sign. The inequality operator is represented with either '!=' or '<>' operators:

In [11]:
%%sql

SELECT BillingAddress, BillingCity, Total
FROM invoices
WHERE BillingCity != 'Paris' AND BillingCity <> 'Berlin'
LIMIT 5

 * sqlite:///data/chinook.db
Done.


BillingAddress,BillingCity,Total
Theodor-Heuss-Straße 34,Stuttgart,1.98
Ullevålsveien 14,Oslo,3.96
Grétrystraat 63,Brussels,5.94
8210 111 ST NW,Edmonton,8.91
69 Salem Street,Boston,13.86


### Easier Filtering With BETWEEN And IN

Some similar conditionals are used very often and writing them out with simple booleans becomes cumbersome. For example, Pandas has `.isin()` function which checks if a value belongs to a list of groups or values. If we wanted to select all invoices for 5 cities, we would have to write 5 chained conditions. Luckily, SQL supports a similar IN operator like `.isin()`:

In [12]:
%%sql

SELECT * 
  FROM invoices
 WHERE BillingCity IN ('Berlin', 'Paris', 'New York', 'Boston', 'London')
 LIMIT 5
    
--invoices[invoices['BillingCity'].isin(
--      ('Berlin', 'Paris', 'New York', 'Boston', 'London')
--)]

 * sqlite:///data/chinook.db
Done.


InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
8,40,2009-02-01 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
11,52,2009-02-06 00:00:00,202 Hoxton Street,London,None,United Kingdom,N1 5LH,8.91
19,40,2009-03-14 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,13.86


The list of values after IN should be given as a tuple, not as a list. You can also negate the condition with the NOT keyword:

In [13]:
%%sql

SELECT * 
  FROM invoices
 WHERE BillingCity NOT IN ('Berlin', 'Paris', 'New York', 'Boston', 'London')
 LIMIT 5
    
--invoices[~invoices['BillingCity'].isin(
--    ('Berlin', 'Paris', 'New York', 'Boston', 'London')
--)]

 * sqlite:///data/chinook.db
Done.


InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99


Another common filtering operation on numeric columns is to select values within a range. For this, BETWEEN keyword can be used, which is equivalent to `pd.Series.between()`:

In [14]:
%%sql

SELECT BillingCity, Total
FROM invoices
WHERE Total BETWEEN 5 AND 15
LIMIT 5

--invoices[invoices['Total'].between(5, 15)]

 * sqlite:///data/chinook.db
Done.


BillingCity,Total
Brussels,5.94
Edmonton,8.91
Boston,13.86
Dublin,5.94
London,8.91


### Checking For Nulls

Every data source has missing values and databases are no exception. Just like there are a number of ways to explore the missing values in Pandas, there are certain keywords that check the existence of null values in SQL. The below query counts the number of rows there are missing values in BillingState:

In [15]:
%%sql

SELECT COUNT(*) FROM
invoices
WHERE BillingState IS NULL

--invoices['BillingState'].isnull().sum()

 * sqlite:///data/chinook.db
Done.


COUNT(*)
202


You can add the NOT keyword between IS and NULL to filter out missing values of a particular column:

In [16]:
%%sql

SELECT InvoiceDate, BillingCountry 
FROM invoices
WHERE Total IS NOT NULL
LIMIT 10

 * sqlite:///data/chinook.db
Done.


InvoiceDate,BillingCountry
2009-01-01 00:00:00,Germany
2009-01-02 00:00:00,Norway
2009-01-03 00:00:00,Belgium
2009-01-06 00:00:00,Canada
2009-01-11 00:00:00,USA
2009-01-19 00:00:00,Germany
2009-02-01 00:00:00,Germany
2009-02-01 00:00:00,France
2009-02-02 00:00:00,France
2009-02-03 00:00:00,Ireland


### Better String Matching With LIKE

In the WHERE clause, we were able to filter columns based on exact text values. But often, we may want to filter textual columns based on a pattern. In Pandas and pure Python, we would use regular expressions for pattern matching, which are very powerful but requires time to master. 

As an alternative, SQL offers '%' wildcard as a placeholder to mach any character 0 or more times. For example, 'gr%' string matches 'great', 'groom', 'greed' and '%ex%' matches any text that has 'ex' in the middle, etc. Let's see how to use it with SQL:

In [17]:
%%sql

SELECT Name, Composer, UnitPrice 
  FROM tracks
 WHERE Name LIKE 'B%'
 LIMIT 5

--tracks[tracks['Name'].str.startswith('B')]

 * sqlite:///data/chinook.db
Done.


Name,Composer,UnitPrice
Balls to the Wall,None,0.99
Breaking The Rules,"Angus Young, Malcolm Young, Brian Johnson",0.99
Bad Boy Boogie,AC/DC,0.99
Blind Man,"Steven Tyler, Joe Perry, Taylor Rhodes",0.99
Bleed The Freak,Jerry Cantrell,0.99


Above query finds all sons that starts with 'B'. The string that contains the wildcard should come after the LIKE keyword.

Now, let's find all songs that contain the word 'beautiful' in their titles:

In [18]:
%%sql

SELECT Name, Composer, UnitPrice
  FROM tracks
 WHERE Name LIKE '%beautiful%'

--tracks[tracks['Name'].str.contains('beautiful')]

 * sqlite:///data/chinook.db
Done.


Name,Composer,UnitPrice
So Beautiful,Mick Hucknall,0.99
Beautiful Day,"Adam Clayton, Bono, Larry Mullen, The Edge",0.99
Beautiful Boy,None,0.99
On the Beautiful Blue Danube,Johann Strauss II,0.99


You can also use other boolean operators next to LIKE:

In [19]:
%%sql

SELECT Name, Composer, UnitPrice
  FROM tracks
 WHERE (Name LIKE 'F%') AND (UnitPrice > 1.0)

--tracks[(tracks['Name'].str.startswith('F')) & (tracks['UnitPrice'] > 1.0)]

 * sqlite:///data/chinook.db
Done.


Name,Composer,UnitPrice
Fallout,None,1.99
Further Instructions,None,1.99
Flashes Before Your Eyes,None,1.99
Fire + Water,None,1.99
Five Years Gone,None,1.99
Fire In Space,None,1.99


There are many other wildcards in SQL that behave similarly. You can see the full list and their usage [here](https://www.w3schools.com/sql/sql_wildcards.asp).

### Aggregate Functions in SQL

It is also possible to perform basic arithmetic operations on columns. These operations are called aggregate functions in SQL and the most common ones are `AVG, SUM, MIN, MAX`. Their functionality should be from their names:

In [21]:
%%sql 

SELECT SUM(Total), MAX(Total), MIN(Total), AVG(Total)
FROM invoices

 * sqlite:///data/chinook.db
Done.


SUM(Total),MAX(Total),MIN(Total),AVG(Total)
2328.600000000004,25.86,0.99,5.651941747572825


Aggregate functions give only a single result for the column they were used on. This means you cannot aggregate across one column and select other unaggregated columns:

In [26]:
%%sql

SELECT AVG(Total), BillingCity, BillingAddress
FROM invoices;

-- the result will be a single row because
-- of the presence of aggregate functions

 * sqlite:///data/chinook.db
Done.


AVG(Total),BillingCity,BillingAddress
5.651941747572825,Stuttgart,Theodor-Heuss-Straße 34


You can combine aggregate functions with conditionals using WHERE clauses just as easily:

In [28]:
%%sql

SELECT AVG(Total), BillingCity
FROM invoices
WHERE BillingCity = 'Paris'

-- invoices[invoices['BillingCity']]['Total'].mean()

 * sqlite:///data/chinook.db
Done.


AVG(Total),BillingCity
5.517142857142857,Paris


It is also possible to use arithmetic operators such as +, -, \*, / on columns and on simple numbers. When used on columns, the operation is performed element-wise:

In [36]:
%%sql

SELECT SUM(Total) / COUNT(Total)
FROM invoices

-- finding the mean manually

 * sqlite:///data/chinook.db
Done.


SUM(Total) / COUNT(Total)
5.651941747572825


One thing to note about arithmetic operations: if you perform operations on only integers, SQL thinks that you are expecting an integer as the answer:

In [48]:
%%sql 

SELECT 10 / 3

 * sqlite:///data/chinook.db
Done.


10 / 3
3


Instead of returning 3.33..., the result is 3. To get a float result, you should use at least one float in the query or use all floats to be safe:

In [49]:
%%sql 

SELECT 10.0 / 3.0

 * sqlite:///data/chinook.db
Done.


10.0 / 3.0
3.3333333333333335


Using this knowledge, let's calculate the average duration of a song in minutes:

In [50]:
%%sql

SELECT Milliseconds / 1000.0 / 60.0
FROM tracks
LIMIT 10

 * sqlite:///data/chinook.db
Done.


Milliseconds / 1000.0 / 60.0
5.72865
5.709366666666667
3.84365
4.20085
6.256966666666667
3.4277
3.8987666666666665
3.5139
3.3850333333333333
4.391616666666667


If you pay attention to the above result, the column name is written as the query that was used to generate that column. When using large calculations, such as finding the standard deviation or variance of a column, this can be an issue because the column name will be as large as the query itself. 

To avoid this, SQL allows aliasing which is similar to aliasing import statements in Python. For example:

In [51]:
%%sql

SELECT SUM(Total) as sum_total
FROM invoices

 * sqlite:///data/chinook.db
Done.


sum_total
2328.600000000004


Using `as` keyword after a single item in a `SELECT` statement tells SQL that we are aliasing. Here are more examples:

In [52]:
%%sql

SELECT SUM(Total) as sum_total,
       MAX(Total) as max_total,
       MIN(Total) as min_total,
       SUM(Total) / COUNT(Total) as mean_total
FROM invoices;

 * sqlite:///data/chinook.db
Done.


sum_total,max_total,min_total,mean_total
2328.600000000004,25.86,0.99,5.651941747572825


You can use aliasing just as easily for simple columns with long names. 

### Ordering Results in SQL

Just like Pandas has `sort_values` method, SQL supports ordering columns via `ORDER BY` clause. Passing a column name after it sorts the results in ascending order:

In [59]:
%%sql

SELECT Name, Composer, UnitPrice
FROM tracks
WHERE Composer <> 'None'
ORDER BY Composer
LIMIT 10

-- tracks.sort_values('Compose')[['Name', 'Compose', 'UnitPrice']]

 * sqlite:///data/chinook.db
Done.


Name,Composer,UnitPrice
Iron Man,"A. F. Iommi, W. Ward, T. Butler, J. Osbourne",0.99
Children Of The Grave,"A. F. Iommi, W. Ward, T. Butler, J. Osbourne",0.99
Paranoid,"A. F. Iommi, W. Ward, T. Butler, J. Osbourne",0.99
New Rhumba,A. Jamal,0.99
Astronomy,A.Bouchard/J.Bouchard/S.Pearlman,0.99
Hard To Handle,A.Isbell/A.Jones/O.Redding,0.99
Go Down,AC/DC,0.99
Dog Eat Dog,AC/DC,0.99
Let There Be Rock,AC/DC,0.99
Bad Boy Boogie,AC/DC,0.99


We are ordering the tracks table in ascending order by the composer name. Note that ORDER BY statement should always come after the WHERE clause. It is also possible to pass 2 or more columns to ORDER BY:

In [61]:
%%sql

SELECT Name, Composer, UnitPrice
FROM tracks
WHERE Composer <> 'None'
ORDER BY UnitPrice, Composer, Name
LIMIT 10

-- tracks.sort_values(['UnitPrice', 'Composer', 'Name'])[['Name', 'Compose', 'UnitPrice']]

 * sqlite:///data/chinook.db
Done.


Name,Composer,UnitPrice
Children Of The Grave,"A. F. Iommi, W. Ward, T. Butler, J. Osbourne",0.99
Iron Man,"A. F. Iommi, W. Ward, T. Butler, J. Osbourne",0.99
Paranoid,"A. F. Iommi, W. Ward, T. Butler, J. Osbourne",0.99
New Rhumba,A. Jamal,0.99
Astronomy,A.Bouchard/J.Bouchard/S.Pearlman,0.99
Hard To Handle,A.Isbell/A.Jones/O.Redding,0.99
Bad Boy Boogie,AC/DC,0.99
Dog Eat Dog,AC/DC,0.99
Go Down,AC/DC,0.99
Hell Ain't A Bad Place To Be,AC/DC,0.99


You can also reverse the ordering by passing the `DESC` keyword after each column name:

In [66]:
%%sql

SELECT Name, Composer, UnitPrice
FROM tracks
WHERE Composer != 'None'
ORDER BY UnitPrice DESC, Composer DESC, Name ASC
LIMIT 10

-- tracks.sort_values(['UnitPrice', 'Composer', 'Name'])[['Name', 'Compose', 'UnitPrice']]

 * sqlite:///data/chinook.db
Done.


Name,Composer,UnitPrice
A Twist In The Tail,roger glover,0.99
Lick It Up,roger glover,0.99
One Man's Meat,roger glover,0.99
Ramshackle Man,roger glover,0.99
Solitaire,roger glover,0.99
Talk About Love,roger glover,0.99
Time To Kill,roger glover,0.99
Loves Been Good To Me,rod mckuen,0.99
For Once In My Life,orlando murden/ronald miller,0.99
The Lady Is A Tramp,lorenz hart/richard rodgers,0.99


The above query returns 3 columns after ordering the UnitPrice and Compose in descending order and the name in ascending order (`ASC` is a default keyword).

### Grouping in SQL

One of the most powerful functions in Pandas is the `groupby`. It can be used to transform a table into virtually any shape you want. Its very close cousin in SQL - `GROUP BY` clause can be used to achieve the same functionality. For example, below query counts the number of songs in each genre:

In [73]:
%%sql

SELECT GenreID, COUNT(*) as genre_count
FROM tracks
GROUP BY GenreId
LIMIT 10

-- tracks.groupby('GenreId')['GenreID'].count()

 * sqlite:///data/chinook.db
Done.


GenreId,genre_count
1,1297
2,130
3,374
4,332
5,12
6,81
7,579
8,58
9,48
10,43


The difference between the GROUP BY in SQL and `groupby` in Pandas is that SQL does not allow selecting columns which weren't given in the GROUP BY clause. For example, adding an extra free column in the above query generates an error:

```sql
%%sql

SELECT Name, GenreID, COUNT(*) as genre_count
FROM tracks
GROUP BY GenreId

syntax error at or near "GROUP"
LINE 1: GROUP BY GenreId;
```



However, you can choose as many columns in the SELECT statement as you like as long as you are using some type of aggregate function on them:

In [86]:
%%sql

SELECT GenreId, AlbumId, 
       COUNT(*) as genre_count,
       AVG(Milliseconds) / 1000.0 / 60.0 as avg_duration,
       AVG(UnitPrice) as avg_price
  FROM tracks
 GROUP BY GenreId, AlbumId
 LIMIT 10

 * sqlite:///data/chinook.db
Done.


GenreId,AlbumId,genre_count,avg_duration,avg_price
1,1,10,4.0006916666666665,0.99
1,2,1,5.709366666666667,0.99
1,3,3,4.767155555555556,0.9899999999999999
1,4,8,5.11095625,0.9900000000000001
1,5,15,4.90189888888889,0.9900000000000001
1,6,13,4.424262820512821,0.9900000000000001
1,7,12,4.513006944444444,0.9900000000000001
1,10,14,4.67584880952381,0.9900000000000001
1,30,14,5.345142857142857,0.9900000000000001
1,31,9,4.566535185185185,0.99


The above query includes almost all the topics we have learned up to this point. We are grouping by both album ID and genre ID and for each group, we are calculating average duration and price of a song. We are also making an efficient use of aliasing. 

We can make the above query even more powerful by ordering by the average duration and genre count:

In [88]:
%%sql

SELECT GenreId, AlbumId, 
       COUNT(*) as genre_count,
       AVG(Milliseconds) / 1000.0 / 60.0 as avg_duration,
       AVG(UnitPrice) as avg_price
  FROM tracks
 GROUP BY GenreId, AlbumId
 ORDER BY avg_duration DESC, genre_count DESC
 LIMIT 10

 * sqlite:///data/chinook.db
Done.


GenreId,AlbumId,genre_count,avg_duration,avg_price
19,227,5,52.615260000000006,1.9899999999999998
20,253,24,48.759572222222225,1.9900000000000002
20,227,2,45.771458333333335,1.99
21,229,22,45.60556136363636,1.99
19,231,16,44.05144166666666,1.9899999999999993
18,227,12,43.763733333333334,1.9899999999999995
21,231,8,43.750514583333334,1.99
18,226,1,43.704166666666666,1.99
19,229,4,43.6093375,1.99
19,228,3,43.44713888888889,1.99


If you pay attention, we are using the alias names of the aggregate functions in the ORDER BY clause. Once you alias a column or the result of the aggregate function, you can refer to them only by their alias for the rest of the query.

### Using conditionals with HAVING

By default, SQL does not allow conditional filtering using aggregate functions in the WHERE clause. For example, we want to select only the genres where the number of songs are greater than 100. Let's try this with WHERE clause:

```sql
%%sql

SELECT GenreId
FROM tracks
GROUP BY GenreId
WHERE COUNT(GenreId) > 10

syntax error at or near "WHERE"
LINE 1: where COUNT(GenreId) > 10;
```

The correct way of filtering rows based on the results of aggregate functions is using the HAVING clause:

In [94]:
%%sql

SELECT GenreId
FROM tracks
GROUP BY GenreId
HAVING COUNT(GenreId) > 100

 * sqlite:///data/chinook.db
Done.


GenreId
1
2
3
4
7


HAVING clause is usually used with GROUP BY. So, whenever you want to filter rows based using aggregate functions, HAVING is the way to go!